In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Fri Aug 15 23:22:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             51W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [108]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("architsharma01/loan-approval-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/loan-approval-prediction-dataset


In [109]:
import pandas as pd
import seaborn as sns
pathDF = "/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv"

df = pd.read_csv(pathDF)
df


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   loan_id                    4269 non-null   int64 
 1    no_of_dependents          4269 non-null   int64 
 2    education                 4269 non-null   object
 3    self_employed             4269 non-null   object
 4    income_annum              4269 non-null   int64 
 5    loan_amount               4269 non-null   int64 
 6    loan_term                 4269 non-null   int64 
 7    cibil_score               4269 non-null   int64 
 8    residential_assets_value  4269 non-null   int64 
 9    commercial_assets_value   4269 non-null   int64 
 10   luxury_assets_value       4269 non-null   int64 
 11   bank_asset_value          4269 non-null   int64 
 12   loan_status               4269 non-null   object
dtypes: int64(10), object(3)
memory usage: 433.7+ KB


In [131]:
df.columns = df.columns.str.strip()

In [132]:
df.corr(numeric_only=True)

,loan_id,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value
loan_id,1.000000,-0.001746,0.085856,0.083929,-0.037429,-0.053055,0.097312,0.104225,0.064667,0.062723
no_of_dependents,-0.001746,1.000000,0.124830,0.103615,-0.037688,-0.027709,0.088878,0.060466,0.085199,0.079248
income_annum,0.085856,0.124830,1.000000,0.929337,-0.054702,0.026845,0.622438,0.652751,0.935610,0.859927
loan_amount,0.083929,0.103615,0.929337,1.000000,-0.054938,0.022682,0.608831,0.621121,0.873936,0.801313
loan_term,-0.037429,-0.037688,-0.054702,-0.054938,1.000000,0.007632,0.022547,-0.031918,-0.036215,-0.005423
cibil_score,-0.053055,-0.027709,0.026845,0.022682,0.007632,1.000000,0.083645,-0.007307,0.010903,0.003375
residential_assets_value,0.097312,0.088878,0.622438,0.608831,0.022547,0.083645,1.000000,0.446839,0.624542,0.551426
commercial_assets_value,0.104225,0.060466,0.652751,0.621121,-0.031918,-0.007307,0.446839,1.000000,0.630533,0.527711
luxury_assets_value,0.064667,0.085199,0.935610,0.873936,-0.036215,0.010903,0.624542,0.630533,1.000000,0.796719
bank_asset_value,0.062723,0.079248,0.859927,0.801313,-0.005423,0.003375,0.551426,0.527711,0.796719,1.000000


In [133]:
for col in df.columns:
    print('\t%s: %d' % (col,df[col].isna().sum()))

	loan_id: 0
	no_of_dependents: 0
	education: 0
	self_employed: 0
	income_annum: 0
	loan_amount: 0
	loan_term: 0
	cibil_score: 0
	residential_assets_value: 0
	commercial_assets_value: 0
	luxury_assets_value: 0
	bank_asset_value: 0
	loan_status: 0


In [134]:
print(df.dtypes)

loan_id                      int64
no_of_dependents             int64
education                   object
self_employed               object
income_annum                 int64
loan_amount                  int64
loan_term                    int64
cibil_score                  int64
residential_assets_value     int64
commercial_assets_value      int64
luxury_assets_value          int64
bank_asset_value             int64
loan_status                 object
dtype: object


In [138]:
numeric_features = ['no_of_dependents', 'income_annum','loan_amount','loan_term','cibil_score','residential_assets_value','commercial_assets_value','luxury_assets_value','bank_asset_value']
categorical_features = ['education','self_employed']


onehot_pipeline = Pipeline([
        ('onehot_encoder', OneHotEncoder(
            handle_unknown='ignore',
            sparse_output=False,
        ))
    ])


numeric_pipeline = Pipeline([
        ('scaler', StandardScaler())
    ])


preprocessor = ColumnTransformer([
        ('cat', onehot_pipeline, categorical_features),
        ('num', numeric_pipeline, numeric_features)
    ], remainder='drop')


In [136]:
df['loan_status'].value_counts()

,count
loan_status,
Approved,264
Rejected,163


In [139]:


X = df.drop('loan_status', axis = 1)
y = df['loan_status']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.2, random_state=42, stratify=y)

preprocessor.fit(X_train, y_train)

X_train_encoded = preprocessor.transform(X_train)
X_test_encoded = preprocessor.transform(X_test)



In [140]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [141]:
df['loan_status'].value_counts()

,count
loan_status,
Approved,264
Rejected,163


In [148]:
print(X_train_encoded.shape)
print(X_test_encoded.shape)

(341, 13)
(86, 13)


In [166]:

param_grid = {
   'kernel': ['rbf','linear'],
    'C': [0.1, 1, 10],
    'gamma': ['auto', 'scale'],
   'class_weight': ['balanced']
}
grid_search = GridSearchCV(SVC(), param_grid, cv = 5)
grid_search.fit(X_train_encoded, y_train_encoded)
print(f"Best SVC model is: {grid_search.best_estimator_}")
print(f"Best SVC model score is: {grid_search.best_score_}")
best_svc = grid_search.best_estimator_

Best SVC model is: SVC(C=0.1, class_weight='balanced', gamma='auto', kernel='linear')
Best SVC model score is: 0.8972719522591646


In [177]:
param_grid= {
    'loss': ['hinge', 'log_loss', 'modified_huber','squared_error'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
}
sgd_clf_cv = GridSearchCV(SGDClassifier(random_state= 42,max_iter= 100000),param_grid, cv = 3)
sgd_clf_cv.fit(X_train_encoded, y_train_encoded)
print(f"Best SGD model is: {sgd_clf_cv.best_estimator_}")
print(f"Best SGD model score is: {sgd_clf_cv.best_score_}")
best_sgd = sgd_clf_cv.best_estimator_

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic

Best SGD model is: SGDClassifier(alpha=0.01, loss='log_loss', max_iter=100000, random_state=42)
Best SGD model score is: 0.9059676044330776


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [183]:
param_grid= {
    'alpha': [0.0001, 0.001, 0.01, .1],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'class_weight': ['balanced']
}
ridge_clf_cv = GridSearchCV(RidgeClassifier(),param_grid, cv = 3)
ridge_clf_cv.fit(X_train_encoded, y_train_encoded)
print(f"Best Ridge model is: {ridge_clf_cv.best_estimator_}")
print(f"Best Ridge model score is: {ridge_clf_cv.best_score_}")
best_ridge = ridge_clf_cv.best_estimator_

Best Ridge model is: RidgeClassifier(alpha=0.0001, class_weight='balanced')
Best Ridge model score is: 0.8944004554158257


In [168]:
param_grid = {
    'criterion':['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']
}

dt_clf_cv = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 5)
dt_clf_cv.fit(X_train_encoded, y_train_encoded)
print(f"Best Decision Tree model is: {dt_clf_cv.best_estimator_}")
print(f"Best Decision Tree model score is: {dt_clf_cv.best_score_}")
best_dt = dt_clf_cv.best_estimator_

Best Decision Tree model is: DecisionTreeClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=4, min_samples_split=10)
Best Decision Tree model score is: 0.9500426257459507
